In [ ]:
import openai
import base64
import os
from tqdm import tqdm
from openai import OpenAI
import random
import numpy as np
import sys
from datetime import datetime
import csv 
from utils import info
from utils import evaluation
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix


def format_causality_prompt(system_info, prompt_text, image_base64_list):
    # Construct messages
    prompt_text = prompt_composition(prompt_text)
    messages = [
        {"role": "system", "content": system_info},
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt_text}
            ] + [
                {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{img_base64}"}}
                for img_base64 in image_base64_list
            ]
        }
    ]
    
    text_prompt = system_info + prompt_text
    return messages, text_prompt


def get_causal_matrix(client, image_base64_list, prompt_type = None, prompt_info = None, dump = False,
                      multi_view = False, 
                      scene_name = None,
                      view = None,
                      images = None,
                      syn_background = False):
    """
    Makes an API call to determine causal relationships in seesaw system images
    Returns a parsed adjacency matrix
    """
    # Construct the messages list with optimized prompt
    if prompt_type == "explicted":
      system_info = "You are a causal discovery expert. Your objective is to analyze the provided images and identify any causal relationships between the variables. Use the identified relationships to complete the causality adjacency matrix and provide a brief explanation supporting your conclusions."
    elif prompt_type == "basic":
      system_info = "Analyze the provided images and identify causal relationships between the variables. Complete the causality adjacency matrix based on the identified relationships and briefly explain your conclusions"
    messages, text_prompt = format_causality_prompt(system_info, prompt_info, image_base64_list)
    # raise ValueError("1111Matrix not found in the response")
    # Make the API call
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        max_tokens=500
    )

    if dump:

        os.makedirs(f"causal_OpenAI_res/{scene_name}", exist_ok=True)
        res = {
          "timestamp": datetime.now().strftime("%Y%m%d_%H%M%S"),
          "prompt_type": prompt_type,
          "images": images,
          "message": text_prompt,
          "response": response.choices[0].message.content,
          "matrix": extract_matrix(response.choices[0].message.content)
  
        }
        file_path = f"causal_OpenAI_res/{scene_name}/{prompt_type}.csv"
        if multi_view:
          file_path = f"causal_OpenAI_res/{scene_name}/{prompt_type}_multi_view_{view}.csv"
        if syn_background:
          file_path = f"causal_OpenAI_res/{scene_name}/{prompt_type}_synbackground_{view}.csv"
        file_exists = os.path.exists(file_path)
        with open(file_path, mode="a", newline="", encoding="utf-8") as f:
                writer = csv.DictWriter(f, fieldnames=["timestamp", "prompt_type", "images","message", "response",'matrix'])
                
                # Write header if the file is new
                if not file_exists:
                    writer.writeheader()
                
                # Write the data row
                writer.writerow(res)
    return extract_matrix(response.choices[0].message.content)

def extract_matrix(text):
  first_index = text.find("```")
  final_index = text.rfind("```")
  # print(text[first_index+3:final_index])
  if first_index == -1 or final_index == -1:
    raise ValueError("Matrix not found in the response")
    return None
  matrix = eval(text[first_index+3:final_index])
  return matrix

def parse_matrix_response(response_text):
    """
    Extracts and validates the adjacency matrix from API response
    Returns numpy array for easier manipulation
    """
    try:
        # Find the matrix pattern
        start_idx = response_text.find('adjacency_matrix = ')
        end_idx = response_text.find(']\n') + 2
        # print(start_idx, end_idx)
        if start_idx == -1 or end_idx == -1:
            raise ValueError("Matrix not found in response")
            
        matrix_str = response_text[start_idx:end_idx]
        # Convert to Python list and validate
        matrix_str = matrix_str.replace('adjacency_matrix = ', '')
        matrix = eval(matrix_str)
        
        # Validation checks
        if not isinstance(matrix, list) or len(matrix) != 5:
            raise ValueError("Invalid matrix dimensions")
            
        if not all(len(row) == 5 for row in matrix):
            raise ValueError("Inconsistent row lengths")
            
        if not all(all(x in {0, 1} for x in row) for row in matrix):
            raise ValueError("Invalid entries: must be 0 or 1")
            
        return np.array(matrix)
        
    except Exception as e:
        raise Exception(f"Matrix parsing failed: {str(e)}")
      

# scene = info.scene()
# scene_info_dict = scene.get_scene("Magnets")
# print(scene_info_dict)
# magnet_img_path = "/home/lds/github/Causality-informed-Generation/code1/database/final_dataset/real/Real_magnet_v3_256P"

# files = os.listdir(magnet_img_path)
# files = [magnet_img_path + file for file in files]

  
def compose_content(dict_info):
  num_of_v = len(dict_info["variables"])
  variables = dict_info["variables"]
  content = ''
  for i,v in enumerate(variables):
    content += f"{i+1}. {variables[v]}\n"
  content = f"There are {num_of_v} variables: \n{content}.\n" 
  content += "Please fill this causality adjacency matrix:\n"
  return content


def prompt_composition(scene_info_dict):
  matrix = scene_info_dict['adjacency_matrix']
  matrix = str(matrix).replace("1", "_,").replace("0", "_,").replace("_,]", '_]')
  matrix = matrix.replace("_]", "_],")
  matrix = '```\n' + matrix
  matrix += '\n```'
  matrix_info = ".\nIn the matrix, matrix[i][j] = 1 means variable i causes variable j, matrix[i][j] = 0 means there is not direct causal relationship."
  scene_info = compose_content(scene_info_dict)
  prompt = scene_info + matrix + matrix_info
  return prompt


def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def get_images(image_dir, img_num, view = -1):
  files = os.listdir(image_dir)
  files = [os.path.join(image_dir, f) for f in files]
  if  view == -1 :
    files = random.sample(files, img_num)
  else:
    # get all of file in files end with view{view}.png
    
    files = [f for f in files if f.endswith(f"_{view}.png")]
    files = random.sample(files, img_num)

  encoded_files = [encode_image(f) for f in files]
  return  files,encoded_files

def calculate_metrics(ground_truth_matrix, predicted_matrices):
    """
    Calculates average accuracy, precision, recall, F1 score, TPR, FPR, and SHD 
    across multiple predicted matrices compared to the ground truth matrix.

    Args:
        ground_truth_matrix (numpy.ndarray): Ground truth binary matrix.
        predicted_matrices (list of numpy.ndarray): List of predicted binary matrices.

    Returns:
        dict: A dictionary containing average metrics.
    """
    metrics = {
        "accuracy": [],
        "precision": [],
        "f1": [],
        "tpr": [],
        "fpr": [],
        "shd": []
    }

    ground_truth_flat = ground_truth_matrix.flatten()

    for predicted_matrix in predicted_matrices:
        predicted_matrix = np.array(predicted_matrix)
        predicted_flat = predicted_matrix.flatten()

        # Calculate confusion matrix components
        tn, fp, fn, tp = confusion_matrix(ground_truth_flat, predicted_flat, labels=[0, 1]).ravel()

        # Accuracy
        accuracy = (tp + tn) / (tp + tn + fp + fn)
        metrics["accuracy"].append(accuracy)

        # Precision
        precision = precision_score(ground_truth_flat, predicted_flat)
        metrics["precision"].append(precision)

        # Recall (True Positive Rate - TPR)
        recall = recall_score(ground_truth_flat, predicted_flat)
        metrics["tpr"].append(recall)

        # F1 Score
        f1 = f1_score(ground_truth_flat, predicted_flat)
        metrics["f1"].append(f1)

        # False Positive Rate (FPR)
        fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
        metrics["fpr"].append(fpr)

        # Structural Hamming Distance (SHD)
        shd = np.sum(ground_truth_matrix != predicted_matrix)
        metrics["shd"].append(shd)

    # Average metrics
    average_metrics = {key: np.mean(value) for key, value in metrics.items()}
    return average_metrics

def main(image_dir, scene_name, strategy, default_image_size = 10, multi_view = False, 
         view = 0, syn_background = False):
    client = OpenAI(api_key=api_key)  # this is also the default, it can be omitted)
    image_dir = image_dir
    scene_name = scene_name
    strategy = strategy
    images, encoded_imgs = get_images(image_dir, default_image_size, view)
    image_base64_list = encoded_imgs
    all_results = []
    scene = info.scene()
    scene_info_dict = scene.get_scene(scene_name)
    for i in tqdm(range(15)):
        try:
            
            matrix = get_causal_matrix(client, image_base64_list, 
                                       prompt_type=strategy,
                                       prompt_info=scene_info_dict, 
                                       dump=True, 
                                       scene_name=scene_name,
                                       multi_view = multi_view,
                                       view = view,
                                       images = images, 
                                       syn_background = syn_background)
            all_results.append(matrix)
        except Exception as e:
            print(f"Error: {str(e)}")
            
    ground_truth = scene_info_dict['adjacency_matrix']
    print(ground_truth)
    print(all_results)
    average_metrics = calculate_metrics(ground_truth, all_results)
    print(average_metrics)
    return average_metrics


---
### Real

In [13]:
main(image_dir = "/home/lds/github/Causality-informed-Generation/code1/database/final_dataset/real/Real_magnet_v3_256P/Real_magnet_v3",
     scene_name = "Magnets",
     strategy = "basic",)

  0%|          | 0/15 [00:00<?, ?it/s]

 53%|█████▎    | 8/15 [01:12<01:00,  8.62s/it]

Error: invalid syntax (<string>, line 2)


100%|██████████| 15/15 [02:04<00:00,  8.31s/it]

[[0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 0]]
[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 1, 1, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]]]


{'accuracy': 0.8660714285714286,
 'precision': 0.9047619047619048,
 'f1': 0.5119047619047619,
 'tpr': 0.3809523809523809,
 'fpr': 0.02197802197802198,
 'shd': 2.142857142857143}

In [14]:
main(image_dir = "/home/lds/github/Causality-informed-Generation/code1/database/final_dataset/real/Real_magnet_v3_256P/Real_magnet_v3",
     scene_name = "Magnets",
     strategy = "explicted",)

  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████| 15/15 [01:46<00:00,  7.11s/it]

[[0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 0]]
[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]]]


{'accuracy': 0.875,
 'precision': 1.0,
 'f1': 0.5,
 'tpr': 0.33333333333333326,
 'fpr': 0.0,
 'shd': 2.0}

---


In [18]:
main(image_dir = "/home/lds/github/Causality-informed-Generation/code1/database/final_dataset/real/Real_spring_v3_256P/Real_spring_v3_256P/",
     scene_name = "Spring",
     strategy = "basic",)

main(image_dir = "/home/lds/github/Causality-informed-Generation/code1/database/final_dataset/real/Real_spring_v3_256P/Real_spring_v3_256P/",
     scene_name = "Spring",
     strategy = "explicted",)

  0%|          | 0/15 [00:00<?, ?it/s]

 13%|█▎        | 2/15 [00:15<01:38,  7.61s/it]

Error: invalid syntax (<string>, line 2)


 47%|████▋     | 7/15 [01:14<01:33, 11.67s/it]

Error: invalid syntax (<string>, line 2)


...

 67%|██████▋   | 10/15 [01:59<00:58, 11.67s/it]

Error: Matrix not found in the response


100%|██████████| 15/15 [02:55<00:00, 11.72s/it]

[[0 0 1]
 [0 0 1]
 [0 0 0]]
[[[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 0], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]]]
{'accuracy': 0.9914529914529915, 'precision': 1.0, 'f1': 0.9743589743589743, 'tpr': 0.9615384615384616, 'fpr': 0.0, 'shd': 0.07692307692307693}


{'accuracy': 0.9914529914529915,
 'precision': 1.0,
 'f1': 0.9743589743589743,
 'tpr': 0.9615384615384616,
 'fpr': 0.0,
 'shd': 0.07692307692307693}

In [12]:
print("multiple View")
main(image_dir = "/home/lds/github/Causality-informed-Generation/code1/database/Real_BACKGROUND_spring_v3/",
     scene_name = "Spring",
     strategy = "basic",
     multi_view = True,
     view = -1)

multiple View
Spring


 87%|████████▋ | 13/15 [01:47<00:20, 10.18s/it]

Error: Matrix not found in the response


100%|██████████| 15/15 [02:00<00:00,  8.03s/it]

[[0 0 1]
 [0 0 1]
 [0 0 0]]
[[[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 1, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 0], [0, 0, 1], [0, 0, 0]], [[0, 0, 0], [0, 0, 1], [0, 0, 1]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]]]
{'accuracy': 0.9682539682539683, 'precision': 0.9404761904761905, 'f1': 0.9261904761904762, 'tpr': 0.9285714285714286, 'fpr': 0.02040816326530612, 'shd': 0.2857142857142857}


{'accuracy': 0.9682539682539683,
 'precision': 0.9404761904761905,
 'f1': 0.9261904761904762,
 'tpr': 0.9285714285714286,
 'fpr': 0.02040816326530612,
 'shd': 0.2857142857142857}

In [13]:
print("multiple View")
main(image_dir = "/home/lds/github/Causality-informed-Generation/code1/database/Real_BACKGROUND_spring_v3/",
     scene_name = "Spring",
     strategy = "basic",
     multi_view = True,
     view = 0)

print("multiple View")
main(image_dir = "/home/lds/github/Causality-informed-Generation/code1/database/Real_BACKGROUND_spring_v3/",
     scene_name = "Spring",
     strategy = "basic",
     multi_view = True,
     view = 2)

print("multiple View")
main(image_dir = "/home/lds/github/Causality-informed-Generation/code1/database/Real_BACKGROUND_spring_v3/",
     scene_name = "Spring",
     strategy = "basic",
     multi_view = True,
     view = 4)

multiple View
Spring


100%|██████████| 15/15 [01:56<00:00,  7.80s/it]


[[0 0 1]
 [0 0 1]
 [0 0 0]]
[[[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], ([[0, 0, 1], [0, 0, 1], [0, 0, 0]],), [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 0], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]]]
{'accuracy': 0.9925925925925926, 'precision': 1.0, 'f1': 0.9777777777777777, 'tpr': 0.9666666666666667, 'fpr': 0.0, 'shd': 0.06666666666666667}
multiple View
Spring


100%|██████████| 15/15 [01:58<00:00,  7.93s/it]


[[0 0 1]
 [0 0 1]
 [0 0 0]]
[[[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 0], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 0], [0, 0, 1], [0, 0, 0]]]
{'accuracy': 0.9851851851851853, 'precision': 1.0, 'f1': 0.9555555555555555, 'tpr': 0.9333333333333333, 'fpr': 0.0, 'shd': 0.13333333333333333}
multiple View
Spring


 47%|████▋     | 7/15 [00:52<01:01,  7.73s/it]

Error: invalid syntax (<string>, line 2)


100%|██████████| 15/15 [01:48<00:00,  7.20s/it]

[[0 0 1]
 [0 0 1]
 [0 0 0]]
[[[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 0], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]]]
{'accuracy': 0.9920634920634921, 'precision': 1.0, 'f1': 0.9761904761904762, 'tpr': 0.9642857142857143, 'fpr': 0.0, 'shd': 0.07142857142857142}


{'accuracy': 0.9920634920634921,
 'precision': 1.0,
 'f1': 0.9761904761904762,
 'tpr': 0.9642857142857143,
 'fpr': 0.0,
 'shd': 0.07142857142857142}

In [14]:
main(image_dir = "/home/lds/github/Causality-informed-Generation/code1/database/Real_BACKGROUND_spring_v3/",
     scene_name = "Spring",
     strategy = "basic",
     multi_view = True,
     view = 6)

main(image_dir = "/home/lds/github/Causality-informed-Generation/code1/database/Real_BACKGROUND_spring_v3/",
     scene_name = "Spring",
     strategy = "basic",
     multi_view = True,
     view = 8)

Spring


 40%|████      | 6/15 [00:40<01:01,  6.78s/it]

Error: invalid syntax (<string>, line 2)


 60%|██████    | 9/15 [01:01<00:40,  6.79s/it]

Error: invalid syntax (<string>, line 2)


...

 93%|█████████▎| 14/15 [01:46<00:07,  7.16s/it]

Error: invalid syntax (<string>, line 2)


100%|██████████| 15/15 [01:54<00:00,  7.65s/it]

[[0 0 1]
 [0 0 1]
 [0 0 0]]
[[[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 0], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]]]
{'accuracy': 0.9920634920634921, 'precision': 1.0, 'f1': 0.9761904761904762, 'tpr': 0.9642857142857143, 'fpr': 0.0, 'shd': 0.07142857142857142}


{'accuracy': 0.9920634920634921,
 'precision': 1.0,
 'f1': 0.9761904761904762,
 'tpr': 0.9642857142857143,
 'fpr': 0.0,
 'shd': 0.07142857142857142}

In [10]:
print("multiple View")
main(image_dir = "/home/lds/github/Causality-informed-Generation/code1/database/Real_BACKGROUND_spring_v3/",
     scene_name = "Spring",
     strategy = "basic",
     multi_view = True,
     view = 7)

multiple View
Spring


 20%|██        | 3/15 [00:22<01:30,  7.57s/it]

Error: Matrix not found in the response


 40%|████      | 6/15 [00:42<01:00,  6.73s/it]

Error: invalid syntax (<string>, line 2)


100%|██████████| 15/15 [01:55<00:00,  7.72s/it]

[[0 0 1]
 [0 0 1]
 [0 0 0]]
[[[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 0], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]]]
{'accuracy': 0.9914529914529915, 'precision': 1.0, 'f1': 0.9743589743589743, 'tpr': 0.9615384615384616, 'fpr': 0.0, 'shd': 0.07692307692307693}


{'accuracy': 0.9914529914529915,
 'precision': 1.0,
 'f1': 0.9743589743589743,
 'tpr': 0.9615384615384616,
 'fpr': 0.0,
 'shd': 0.07692307692307693}

In [25]:
print("multiple View")
path = "/home/lds/github/Causality-informed-Generation/code1/database/Real_BACKGROUND_spring_v3/"
for i in range(10):
  ii = i - 1
  print(f"view {ii}")
  main(image_dir = path,
      scene_name = "Spring",
      strategy = "basic",
      multi_view = True,
      view = ii, syn_background = False)

multiple View
view -1
Spring


100%|██████████| 15/15 [01:57<00:00,  7.86s/it]


[[0 0 1]
 [0 0 1]
 [0 0 0]]
[[[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 0], [0, 0, 1], [1, 1, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 0], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]]]
{'accuracy': 0.9703703703703703, 'precision': 0.9555555555555555, 'f1': 0.9377777777777777, 'tpr': 0.9333333333333333, 'fpr': 0.019047619047619046, 'shd': 0.26666666666666666}
view 0
Spring


100%|██████████| 15/15 [02:03<00:00,  8.25s/it]


[[0 0 1]
 [0 0 1]
 [0 0 0]]
[[[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]]]
{'accuracy': 1.0, 'precision': 1.0, 'f1': 1.0, 'tpr': 1.0, 'fpr': 0.0, 'shd': 0.0}
view 1
Spring


...

[[0 0 1]
 [0 0 1]
 [0 0 0]]
[[[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]]]
{'accuracy': 1.0, 'precision': 1.0, 'f1': 1.0, 'tpr': 1.0, 'fpr': 0.0, 'shd': 0.0}
view 8
Spring


 53%|█████▎    | 8/15 [00:56<00:55,  7.97s/it]

Error: invalid syntax (<string>, line 2)


100%|██████████| 15/15 [01:43<00:00,  6.91s/it]

[[0 0 1]
 [0 0 1]
 [0 0 0]]
[[[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]]]
{'accuracy': 1.0, 'precision': 1.0, 'f1': 1.0, 'tpr': 1.0, 'fpr': 0.0, 'shd': 0.0}


In [22]:
print("multiple View")
path = "/home/lds/github/Causality-informed-Generation/code1/database/Real_spring_v3_256P/"
for i in range(10):
  ii = i - 1
  print(f"view {ii}")
  main(image_dir = path,
      scene_name = "Spring",
      strategy = "basic",
      multi_view = True,
      view = ii, syn_background = True)

multiple View
view -1
Spring


 27%|██▋       | 4/15 [00:27<01:16,  6.92s/it]

Error: Matrix not found in the response


100%|██████████| 15/15 [01:45<00:00,  7.01s/it]


[[0 0 1]
 [0 0 1]
 [0 0 0]]
[[[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 0], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]]]
{'accuracy': 0.9920634920634921, 'precision': 1.0, 'f1': 0.9761904761904762, 'tpr': 0.9642857142857143, 'fpr': 0.0, 'shd': 0.07142857142857142}
view 0
Spring


...

Error: Matrix not found in the response


 93%|█████████▎| 14/15 [01:36<00:06,  6.38s/it]

Error: Matrix not found in the response


100%|██████████| 15/15 [01:43<00:00,  6.88s/it]

[[0 0 1]
 [0 0 1]
 [0 0 0]]
[[[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 0], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]]]
{'accuracy': 0.9907407407407408, 'precision': 1.0, 'f1': 0.9722222222222222, 'tpr': 0.9583333333333334, 'fpr': 0.0, 'shd': 0.08333333333333333}


---
### magnet


In [9]:
path = "/home/lds/github/Causality-informed-Generation/code1/database/Real_magnet_v3_blank_floor/"
for i in range(9):
  ii = i - 1
  print(f"view {ii}")
  main(image_dir = path,
      scene_name = "Magnets",
      strategy = "basic",
      multi_view = True,
      view = ii, syn_background = False)

view -1
Magnets


100%|██████████| 15/15 [03:27<00:00, 13.84s/it]


[[0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 0]]
[[[0, 0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 1], [1, 1, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]]]
{'accuracy': 0.8583333333333333, 'precision': 0.8577777777777776, 'f1': 0.5277777777

 13%|█▎        | 2/15 [00:19<02:06,  9.76s/it]

Error: invalid syntax (<string>, line 2)


...

Error: invalid syntax (<string>, line 2)


100%|██████████| 15/15 [02:42<00:00, 10.85s/it]


[[0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 0]]
[[[0, 0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [1, 1, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [1, 1, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]]]
{'accuracy': 0.8557692307692307, 'precision': 0.7948717948717949, 'f1': 0.5256410256410257, 'tpr': 0.4358974358974358, 'fpr': 0.04733727810650888, 'shd': 2.3076923076923075}
view 7
Magnets


100%|██████████| 15/15 [02:20<00:00,  9.35s/it]

[[0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 0]]
[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 1, 1, 0], [1, 0, 1, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 1], [1, 1, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]]]
{'accuracy': 0.825, 'precision': 0.8044444444444444, 'f1': 0.4444444444444444, 'tpr'

In [8]:
print("multiple View")
path = "//home/lds/github/Causality-informed-Generation/code1/database/Real_magnet_v3_realistic_back/"
for i in range(0, 2):
  ii = i - 1
  print(f"view {ii}")
  main(image_dir = path,
      scene_name = "Magnets",
      strategy = "basic",
      multi_view = True,
      view = ii, syn_background = False)

multiple View
view -1
Magnets


100%|██████████| 15/15 [02:34<00:00, 10.29s/it]


[[0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 0]]
[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [1, 1, 0, 0]]]
{'accuracy': 0.8583333333333333, 'precision': 0.8222222222222223, 'f1': 0.5222222222

 33%|███▎      | 5/15 [01:19<02:09, 12.95s/it]

Error: invalid syntax (<string>, line 2)


 73%|███████▎  | 11/15 [03:33<01:00, 15.09s/it]

Error: invalid syntax (<string>, line 2)


100%|██████████| 15/15 [04:27<00:00, 17.82s/it]

[[0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 0]]
[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [1, 1, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]]]
{'accuracy': 0.8557692307692307, 'precision': 0.8974358974358974, 'f1': 0.47435897435897434, 'tpr': 0.3333333333333333, 'fpr': 0.02366863905325444, 'shd': 2.3076923076923075}


In [7]:
print("multiple View")
path = "//home/lds/github/Causality-informed-Generation/code1/database/Real_magnet_v3_realistic_back/"
for i in range(2, 9):
  ii = i - 1
  print(f"view {ii}")
  main(image_dir = path,
      scene_name = "Magnets",
      strategy = "basic",
      multi_view = True,
      view = ii, syn_background = False)

multiple View
view 1
Magnets


100%|██████████| 15/15 [02:14<00:00,  8.96s/it]


[[0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 0]]
[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]]]
{'accuracy': 0.8666666666666667, 'precision': 0.9111111111111111, 'f1': 0.5111111111

100%|██████████| 15/15 [02:45<00:00, 11.02s/it]


[[0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 0]]
[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 1], [1, 1, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [1, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [1, 1, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]]]
{'accuracy': 0.8458333333333333, 'precision': 0.78, 'f1': 0.51, 'tpr': 0.42222222222

...

[[0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 0]]
[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 1, 0, 1], [1, 1, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 1], [0, 1, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [1, 1, 0, 0]]]
{'accuracy': 0.8169642857142857, 'precision': 0.7023809523809524, 'f1': 0.4431972789115646, 'tpr': 0.3571428571428571, 'fpr': 0.07692307692307

100%|██████████| 15/15 [02:09<00:00,  8.66s/it]


[[0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 0]]
[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]]]
{'accuracy': 0.85, 'precision': 0.8666666666666667, 'f1': 0.4666666666666666, 'tpr':

100%|██████████| 15/15 [02:18<00:00,  9.27s/it]

[[0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 0]]
[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]]]
{'accuracy': 0.8583333333333333, 'precision': 0.8666666666666667, 'f1': 0.5, 'tpr': 

----

### convex

In [19]:
image_path = "/home/lds/github/Causality-informed-Generation/code1/database/final_dataset/real/Real_seesaw_v3_256P/Real_seesaw_v3_256P/"
main(image_dir = image_path,
     scene_name = "Seesaw",
     strategy = "basic",)

main(image_dir = image_path,
     scene_name = "Seesaw",
     strategy = "explicted",)


  0%|          | 0/15 [00:00<?, ?it/s]

 60%|██████    | 9/15 [01:27<00:53,  8.95s/it]

Error: invalid syntax (<string>, line 2)


100%|██████████| 15/15 [02:33<00:00, 10.20s/it]


Error: invalid syntax (<string>, line 2)
[[0 0 1]
 [0 0 1]
 [0 0 0]]
[[[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]]]
{'accuracy': 1.0, 'precision': 1.0, 'f1': 1.0, 'tpr': 1.0, 'fpr': 0.0, 'shd': 0.0}


 60%|██████    | 9/15 [01:35<01:01, 10.31s/it]

Error: invalid syntax (<string>, line 2)


100%|██████████| 15/15 [02:24<00:00,  9.61s/it]

[[0 0 1]
 [0 0 1]
 [0 0 0]]
[[[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]], [[0, 0, 1], [0, 0, 1], [0, 0, 0]]]
{'accuracy': 1.0, 'precision': 1.0, 'f1': 1.0, 'tpr': 1.0, 'fpr': 0.0, 'shd': 0.0}


{'accuracy': 1.0,
 'precision': 1.0,
 'f1': 1.0,
 'tpr': 1.0,
 'fpr': 0.0,
 'shd': 0.0}

In [20]:
img_path = "/home/lds/github/Causality-informed-Generation/code1/database/final_dataset/real/Real_convex_len_v3_512x256/convex_len_render_images/"
main(image_dir = img_path,
     scene_name = "Convex",
     strategy = "basic",)

main(image_dir = img_path,
     scene_name = "Convex",
     strategy = "explicted",)

 60%|██████    | 9/15 [01:55<01:14, 12.34s/it]

Error: Matrix not found in the response


100%|██████████| 15/15 [03:25<00:00, 13.69s/it]


[[0 1 1]
 [0 0 1]
 [0 0 0]]
[[[0, 1, 1], [0, 0, 1], [0, 0, 0]], [[0, 1, 1], [1, 0, 1], [1, 1, 0]], [[0, 1, 1], [1, 0, 1], [0, 0, 0]], [[0, 1, 1], [0, 0, 1], [0, 0, 0]], [[0, 1, 1], [0, 0, 1], [0, 0, 0]], [[0, 1, 1], [0, 0, 1], [0, 0, 0]], [[0, 1, 1], [1, 0, 1], [0, 0, 0]], [[0, 1, 1], [1, 0, 1], [0, 0, 0]], [[0, 1, 1], [1, 0, 1], [0, 0, 0]], [[0, 1, 1], [0, 0, 1], [0, 0, 0]], [[0, 1, 1], [1, 0, 1], [0, 0, 0]], [[0, 1, 1], [0, 0, 1], [0, 0, 0]], [[0, 1, 1], [0, 0, 1], [0, 0, 0]], [[0, 1, 1], [1, 0, 1], [0, 0, 0]]]
{'accuracy': 0.9285714285714286, 'precision': 0.8571428571428571, 'f1': 0.9149659863945578, 'tpr': 1.0, 'fpr': 0.10714285714285716, 'shd': 0.6428571428571429}


 33%|███▎      | 5/15 [01:08<02:28, 14.82s/it]

...

 87%|████████▋ | 13/15 [02:49<00:30, 15.17s/it]

Error: Matrix not found in the response


100%|██████████| 15/15 [03:18<00:00, 13.23s/it]

[[0 1 1]
 [0 0 1]
 [0 0 0]]
[[[0, 1, 1], [0, 0, 1], [0, 0, 0]], [[0, 1, 1], [0, 0, 1], [0, 0, 0]], [[0, 1, 1], [0, 0, 1], [0, 0, 0]], [[0, 1, 1], [0, 0, 1], [0, 0, 0]], [[0, 1, 1], [0, 0, 1], [0, 0, 0]], [[0, 1, 1], [1, 0, 1], [0, 0, 0]], [[0, 1, 1], [0, 0, 1], [0, 0, 0]], [[0, 1, 1], [0, 0, 1], [0, 0, 0]], [[0, 1, 1], [0, 0, 1], [0, 0, 0]], [[0, 1, 1], [0, 0, 1], [0, 0, 0]], [[0, 1, 1], [1, 0, 1], [0, 0, 0]], [[0, 1, 1], [1, 0, 1], [0, 0, 0]]]
{'accuracy': 0.9722222222222223, 'precision': 0.9375, 'f1': 0.9642857142857144, 'tpr': 1.0, 'fpr': 0.041666666666666664, 'shd': 0.25}


{'accuracy': 0.9722222222222223,
 'precision': 0.9375,
 'f1': 0.9642857142857144,
 'tpr': 1.0,
 'fpr': 0.041666666666666664,
 'shd': 0.25}

In [22]:
image_path = "/home/lds/github/Causality-informed-Generation/code1/database/final_dataset/real/Real_reflection_v2__256P/real_rendered_reflection_256P/"
main(image_dir = image_path,
     scene_name = "Reflection",
     strategy = "basic",)

main(image_dir = image_path,
     scene_name = "Reflection",
     strategy = "explicted",)

  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████| 15/15 [01:46<00:00,  7.13s/it]


[[0 1]
 [0 0]]
[[[0, 1], [0, 0]], [[0, 1], [0, 0]], [[0, 1], [0, 0]], [[0, 1], [0, 0]], [[0, 1], [0, 0]], [[0, 1], [0, 0]], [[0, 1], [0, 0]], [[0, 1], [0, 0]], [[0, 1], [0, 0]], [[0, 1], [0, 0]], [[0, 1], [0, 0]], [[0, 1], [0, 0]], [[0, 1], [0, 0]], [[0, 1], [0, 0]], [[0, 1], [0, 0]]]
{'accuracy': 1.0, 'precision': 1.0, 'f1': 1.0, 'tpr': 1.0, 'fpr': 0.0, 'shd': 0.0}


100%|██████████| 15/15 [01:37<00:00,  6.50s/it]

[[0 1]
 [0 0]]
[[[0, 1], [0, 0]], [[0, 1], [0, 0]], [[0, 1], [0, 0]], [[0, 1], [0, 0]], [[0, 1], [0, 0]], [[0, 1], [0, 0]], [[0, 1], [0, 0]], [[0, 1], [0, 0]], [[0, 1], [0, 0]], [[0, 1], [0, 0]], [[0, 1], [0, 0]], [[0, 1], [0, 0]], [[0, 1], [0, 0]], [[0, 1], [0, 0]], [[0, 1], [0, 0]]]
{'accuracy': 1.0, 'precision': 1.0, 'f1': 1.0, 'tpr': 1.0, 'fpr': 0.0, 'shd': 0.0}


{'accuracy': 1.0,
 'precision': 1.0,
 'f1': 1.0,
 'tpr': 1.0,
 'fpr': 0.0,
 'shd': 0.0}

In [17]:
image_path = "/home/lds/github/Causality-informed-Generation/code1/database/final_dataset/real/Real_water_flow_v5_256/Water_flow_scene_render/"
main(image_dir = image_path,
     scene_name = "Waterflow",
     strategy = "basic",)

main(image_dir = image_path,
     scene_name = "Waterflow",
     strategy = "explicted",)

  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████| 15/15 [03:14<00:00, 12.99s/it]


[[0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 0]]
[[[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0

 40%|████      | 6/15 [01:08<01:42, 11.41s/it]

Error: invalid syntax (<string>, line 2)


100%|██████████| 15/15 [02:42<00:00, 10.86s/it]

[[0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 0]]
[[[0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0

{'accuracy': 0.9600000000000001,
 'precision': 0.8880952380952382,
 'f1': 0.8790249433106576,
 'tpr': 0.8928571428571429,
 'fpr': 0.027210884353741496,
 'shd': 1.0}

In [16]:
image_path = "/home/lds/github/Causality-informed-Generation/code1/database/Real_BACKGROUND_WaterFlow/"

print("multiple View")
main(image_dir = image_path,
     scene_name = "Waterflow",
     strategy = "basic",
     multi_view = True,
     view = 8)

multiple View
Waterflow


  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████| 15/15 [04:55<00:00, 19.68s/it]

[[0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 0]]
[[[0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0

{'accuracy': 0.9466666666666667,
 'precision': 0.7914285714285715,
 'f1': 0.8545165945165946,
 'tpr': 0.95,
 'fpr': 0.05396825396825398,
 'shd': 1.3333333333333333}

In [17]:
image_path = "/home/lds/github/Causality-informed-Generation/code1/database/Real_BACKGROUND_WaterFlow/"

for i in range(9):
  ii = i - 1
  print("multiple View, view", ii)
  main(image_dir = image_path,
       scene_name = "Waterflow",
       strategy = "basic",
       multi_view = True,
       view = ii)

multiple View, view -1
Waterflow


100%|██████████| 15/15 [02:18<00:00,  9.23s/it]


[[0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 0]]
[[[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0

100%|██████████| 15/15 [02:21<00:00,  9.43s/it]


[[0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 0]]
[[[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 0, 1], [0, 0, 1, 0

...

[[0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 0]]
[[[0, 0, 1, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 0

100%|██████████| 15/15 [02:13<00:00,  8.90s/it]


[[0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 0]]
[[[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 1, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0

100%|██████████| 15/15 [02:20<00:00,  9.36s/it]

[[0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 0]]
[[[0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0

In [23]:
print("multiple View")
path = "/home/lds/github/Causality-informed-Generation/code1/database/Real_WaterFlow/"
for i in range(10):
  ii = i - 1
  print(f"view {ii}")
  main(image_dir = path,
      scene_name = "Waterflow",
      strategy = "basic",
      multi_view = True,
      view = ii, syn_background = True)

multiple View
view -1
Waterflow


100%|██████████| 15/15 [02:16<00:00,  9.08s/it]


[[0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 0]]
[[[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0

100%|██████████| 15/15 [02:06<00:00,  8.45s/it]


[[0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 0]]
[[[0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 1, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0

...

[[0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 0]]
[[[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [1, 1, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0

100%|██████████| 15/15 [02:11<00:00,  8.75s/it]


[[0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 0]]
[[[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0

100%|██████████| 15/15 [02:01<00:00,  8.09s/it]

[[0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 0]]
[[[0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 0

---

### Parabola

In [2]:
image_path = "/home/lds/github/Causality-informed-Generation/code1/database/final_dataset/real/Real_parabola_v4_512x256/generated_images/"
main(image_dir = image_path,
     scene_name = "Parabola",
     strategy = "basic",)

main(image_dir = image_path,
     scene_name = "Parabola",
     strategy = "explicted",)

NameError: name 'main' is not defined

In [14]:
image_path = "/home/lds/github/Causality-informed-Generation/code1/database/final_dataset/real/Real_parabola_v4_512x256/generated_images/"
main(image_dir = image_path,
     scene_name = "Parabola",
     strategy = "basic",)

main(image_dir = image_path,
     scene_name = "Parabola",
     strategy = "explicted",)

  7%|▋         | 1/15 [00:08<02:05,  8.94s/it]

Error: invalid syntax (<string>, line 2)


100%|██████████| 15/15 [02:43<00:00, 10.88s/it]


[[0 0 1 1]
 [0 0 1 1]
 [0 0 0 0]
 [0 0 0 0]]
[[[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]]]
{'accuracy': 0.9776785714285714, 'precision': 0.9285714285714287, 'f1': 0.9603174603174605, 'tpr': 1.0, 'fpr': 0.02976190476190476, 'shd': 0.3

100%|██████████| 15/15 [02:20<00:00,  9.39s/it]

[[0 0 1 1]
 [0 0 1 1]
 [0 0 0 0]
 [0 0 0 0]]
[[[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]]]
{'accuracy': 0.975, 'precision': 0.92, 'f1': 0.9555555555555556, 'tpr': 1.0, 'fpr': 

{'accuracy': 0.975,
 'precision': 0.92,
 'f1': 0.9555555555555556,
 'tpr': 1.0,
 'fpr': 0.03333333333333333,
 'shd': 0.4}

In [16]:
image_path = "/home/lds/github/Causality-informed-Generation/code1/database/final_dataset/real/Real_pendulum_v5_256P/Real_pendulum/"
main(image_dir = image_path,
     scene_name = "Pendulum",
     strategy = "basic",)

main(image_dir = image_path,
     scene_name = "Pendulum",
     strategy = "explicted",)

  0%|          | 0/15 [00:00<?, ?it/s]

 53%|█████▎    | 8/15 [01:17<01:07,  9.59s/it]

Error: Matrix not found in the response


100%|██████████| 15/15 [02:19<00:00,  9.31s/it]


[[0 0 0 1 1]
 [0 0 0 1 1]
 [0 0 0 1 0]
 [0 0 0 0 1]
 [0 0 0 0 0]]
[[[0, 0, 0, 1, 1], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 1, 1], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 1, 1], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 1, 1], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 1, 1], [0, 0, 0, 1, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 1, 1], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 1, 1], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 1, 1], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 1, 1], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 1, 1], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 1, 1], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0

100%|██████████| 15/15 [02:32<00:00, 10.18s/it]

[[0 0 0 1 1]
 [0 0 0 1 1]
 [0 0 0 1 0]
 [0 0 0 0 1]
 [0 0 0 0 0]]
[[[0, 0, 0, 1, 1], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 1, 1], [0, 0, 0, 1, 1], [0, 0, 0, 1, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 1, 1], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 1, 1], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 1, 1], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 1, 1], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 1, 1], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 1, 1], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 1, 1], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 1, 1], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 1, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0

{'accuracy': 0.8613333333333334,
 'precision': 0.7755555555555557,
 'f1': 0.6664646464646463,
 'tpr': 0.5888888888888889,
 'fpr': 0.052631578947368404,
 'shd': 3.466666666666667}

In [38]:
image_path = "/home/lds/github/Causality-informed-Generation/code1/database/Real_BACKGROUND_Parabola_v4/"

main(image_dir = image_path,
     scene_name = "Pendulum",
     strategy = "basic",
     multi_view = True,
     view = 8)

Pendulum


100%|██████████| 15/15 [05:48<00:00, 23.26s/it]

[[0 0 0 1 1]
 [0 0 0 1 1]
 [0 0 0 1 0]
 [0 0 0 0 1]
 [0 0 0 0 0]]
[[[0, 0, 0, 1, 1], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 1, 1], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 1, 1], [0, 0, 0, 1, 1], [0, 1, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 1, 1], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 1, 1], [0, 0, 0, 1, 1], [0, 1, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 1, 1], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 1, 1], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 1, 1], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 1, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 1, 1], [0, 0, 0, 1, 1], [0, 1, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 1, 1], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0

{'accuracy': 0.8453333333333334,
 'precision': 0.7211111111111111,
 'f1': 0.6444444444444444,
 'tpr': 0.5888888888888889,
 'fpr': 0.0736842105263158,
 'shd': 3.8666666666666667}

In [16]:
image_path = "/home/lds/github/Causality-informed-Generation/code1/database/Real_BACKGROUND_Parabola_v4/"

for i in range(9):
  ii = i-1
  print("multiple View, view ", ii)
  main(image_dir = image_path,
       scene_name = "Parabola",
       strategy = "basic",
       multi_view = True,
       view = ii)


multiple View, view  -1
Parabola


  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████| 15/15 [02:26<00:00,  9.80s/it]


[[0 0 1 1]
 [0 0 1 1]
 [0 0 0 0]
 [0 0 0 0]]
[[[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 1, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]]]
{'accuracy': 0.9375, 'precision': 0.8177777777777778, 'f1': 0.8903703703703705, 'tpr

 80%|████████  | 12/15 [01:45<00:29,  9.84s/it]

...

[[0 0 1 1]
 [0 0 1 1]
 [0 0 0 0]
 [0 0 0 0]]
[[[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]]]
{'accuracy': 0.9666666666666667, 'precision': 0.8933333333333334, 'f1': 0.9407407407

 73%|███████▎  | 11/15 [01:45<00:34,  8.61s/it]

Error: Matrix not found in the response


100%|██████████| 15/15 [02:33<00:00, 10.25s/it]

[[0 0 1 1]
 [0 0 1 1]
 [0 0 0 0]
 [0 0 0 0]]
[[[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 0, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]]]
{'accuracy': 0.9642857142857143, 'precision': 0.8964285714285715, 'f1': 0.9345238095238095, 'tpr': 0.9821428571428571, 'fpr': 0.04166666666666

In [24]:
print("multiple View")
path = "/home/lds/github/Causality-informed-Generation/code1/database/Real_parabola/"
for i in range(10):
  ii = i - 1
  print(f"view {ii}")
  main(image_dir = path,
      scene_name = "Parabola",
      strategy = "basic",
      multi_view = True,
      view = ii, syn_background = True)

multiple View
view -1
Parabola


100%|██████████| 15/15 [02:07<00:00,  8.50s/it]


[[0 0 1 1]
 [0 0 1 1]
 [0 0 0 0]
 [0 0 0 0]]
[[[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]]]
{'accuracy': 0.9916666666666667, 'precision': 0.9733333333333334, 'f1': 0.9851851851

100%|██████████| 15/15 [01:59<00:00,  7.96s/it]


[[0 0 1 1]
 [0 0 1 1]
 [0 0 0 0]
 [0 0 0 0]]
[[[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]]]
{'accuracy': 0.9666666666666667, 'precision': 0.8933333333333336, 'f1': 0.9407407407

...

[[0 0 1 1]
 [0 0 1 1]
 [0 0 0 0]
 [0 0 0 0]]
[[[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 1, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]]]
{'accuracy': 0.9708333333333333, 'precision': 0.9155555555555556, 'f1': 0.9511111111

100%|██████████| 15/15 [02:00<00:00,  8.00s/it]


[[0 0 1 1]
 [0 0 1 1]
 [0 0 0 0]
 [0 0 0 0]]
[[[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]]]
{'accuracy': 0.975, 'precision': 0.9244444444444445, 'f1': 0.957037037037037, 'tpr':

100%|██████████| 15/15 [02:00<00:00,  8.02s/it]

[[0 0 1 1]
 [0 0 1 1]
 [0 0 0 0]
 [0 0 0 0]]
[[[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]]]
{'accuracy': 0.9666666666666667, 'precision': 0.8933333333333334, 'f1': 0.9407407407

In [11]:
image_path = "/home/lds/github/Causality-informed-Generation/code1/database/Real_BACKGROUND_Parabola_v4/"

print("multiple View")
main(image_dir = image_path,
     scene_name = "Parabola",
     strategy = "basic",
     multi_view = True,
     view = 5)

multiple View
Parabola


100%|██████████| 15/15 [05:10<00:00, 20.68s/it]

[[0 0 1 1]
 [0 0 1 1]
 [0 0 0 0]
 [0 0 0 0]]
[[[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 1], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]]]
{'accuracy': 0.9708333333333333, 'precision': 0.9111111111111112, 'f1': 0.9496296296

{'accuracy': 0.9708333333333333,
 'precision': 0.9111111111111112,
 'f1': 0.9496296296296297,
 'tpr': 1.0,
 'fpr': 0.03888888888888888,
 'shd': 0.4666666666666667}

In [12]:
image_path = "/home/lds/github/Causality-informed-Generation/code1/database/Real_BACKGROUND_Parabola_v4/"

print("multiple View")
main(image_dir = image_path,
     scene_name = "Parabola",
     strategy = "basic",
     multi_view = True,
     view = -1)

multiple View
Parabola


 80%|████████  | 12/15 [01:53<00:28,  9.60s/it]

Error: invalid syntax (<string>, line 2)


 93%|█████████▎| 14/15 [02:11<00:09,  9.20s/it]

Error: Matrix not found in the response


100%|██████████| 15/15 [02:21<00:00,  9.47s/it]

[[0 0 1 1]
 [0 0 1 1]
 [0 0 0 0]
 [0 0 0 0]]
[[[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 1], [0, 0, 0, 0]]]
{'accuracy': 0.9423076923076923, 'precision': 0.8205128205128206, 'f1': 0.8991452991452993, 'tpr': 1.0, 'fpr': 0.07692307692307693, 'shd': 0.9230769230769231}


{'accuracy': 0.9423076923076923,
 'precision': 0.8205128205128206,
 'f1': 0.8991452991452993,
 'tpr': 1.0,
 'fpr': 0.07692307692307693,
 'shd': 0.9230769230769231}

---
### Hypothetical

In [5]:
print("Linear")
path = "/home/lds/github/Causality-informed-Generation/code1/database/final_dataset/hypothetic/data/Hypothetic_v2_linear/"
main(image_dir = path,
     scene_name = "V2",
     strategy = "basic",)

main(image_dir = path,
     scene_name = "V2",
     strategy = "explicted",)

print("Nonlinear")
path = '/home/lds/github/Causality-informed-Generation/code1/database/final_dataset/hypothetic/data/Hypothetic_v2_nonlinear/'
main(image_dir = path,
     scene_name = "V2",
     strategy = "basic",)

main(image_dir = path,
     scene_name = "V2",
     strategy = "explicted",)

Linear


  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████| 15/15 [01:41<00:00,  6.74s/it]
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap

[[0 1]
 [0 0]]
[[[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]]]
{'accuracy': 0.75, 'precision': 0.0, 'f1': 0.0, 'tpr': 0.0, 'fpr': 0.0, 'shd': 1.0}


100%|██████████| 15/15 [01:33<00:00,  6.26s/it]
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap

...

[[0 1]
 [0 0]]
[[[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]]]
{'accuracy': 0.75, 'precision': 0.0, 'f1': 0.0, 'tpr': 0.0, 'fpr': 0.0, 'shd': 1.0}


100%|██████████| 15/15 [01:35<00:00,  6.34s/it]

[[0 1]
 [0 0]]
[[[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]]]
{'accuracy': 0.75, 'precision': 0.0, 'f1': 0.0, 'tpr': 0.0, 'fpr': 0.0, 'shd': 1.0}



/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/minicond

{'accuracy': 0.75,
 'precision': 0.0,
 'f1': 0.0,
 'tpr': 0.0,
 'fpr': 0.0,
 'shd': 1.0}

In [7]:
print("Linear")
path = "/home/lds/github/Causality-informed-Generation/code1/database/final_dataset/hypothetic/data/Hypo_v3_v_structure_256/"
main(image_dir = path,
     scene_name = "V3_V",
     strategy = "basic",)

main(image_dir = path,
     scene_name = "V3_V",
     strategy = "explicted",)

print("Nonlinear")
path = '/home/lds/github/Causality-informed-Generation/code1/database/final_dataset/hypothetic/data/Hypothetic_V3_nonlinear_vstructure/'
main(image_dir = path,
     scene_name = "V3_V",
     strategy = "basic",)

main(image_dir = path,
     scene_name = "V3_V",
     strategy = "explicted",)



Linear


  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████| 15/15 [02:09<00:00,  8.67s/it]
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap

[[0 0 1]
 [0 0 1]
 [0 0 0]]
[[[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]]]
{'accuracy': 0.777777777777778, 'precision': 0.0, 'f1': 0.0, 'tpr': 0.0, 'fpr': 0.0, 'shd': 2.0}


100%|██████████| 15/15 [01:39<00:00,  6.63s/it]
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap

...

[[0 0 1]
 [0 0 1]
 [0 0 0]]
[[[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]]]
{'accuracy': 0.777777777777778, 'precision': 0.0, 'f1': 0.0, 'tpr': 0.0, 'fpr': 0.0, 'shd': 2.0}


100%|██████████| 15/15 [01:38<00:00,  6.57s/it]

[[0 0 1]
 [0 0 1]
 [0 0 0]]
[[[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]]]
{'accuracy': 0.777777777777778, 'precision': 0.0, 'f1': 0.0, 'tpr': 0.0, 'fpr': 0.0, 'shd': 2.0}



/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/minicond

{'accuracy': 0.777777777777778,
 'precision': 0.0,
 'f1': 0.0,
 'tpr': 0.0,
 'fpr': 0.0,
 'shd': 2.0}

In [8]:
print("Linear")
path = "/home/lds/github/Causality-informed-Generation/code1/database/final_dataset/hypothetic/data/Hypothetic_v3_fully_connected_linear/"
main(image_dir = path,
     scene_name = "V3_F",
     strategy = "basic",)

main(image_dir = path,
     scene_name = "V3_F",
     strategy = "explicted",)



Linear


100%|██████████| 15/15 [01:50<00:00,  7.35s/it]
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap

[[0 1 1]
 [0 0 1]
 [0 0 0]]
[[[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]]]
{'accuracy': 0.6666666666666665, 'precision': 0.0, 'f1': 0.0, 'tpr': 0.0, 'fpr': 0.0, 'shd': 3.0}


100%|██████████| 15/15 [01:36<00:00,  6.42s/it]

[[0 1 1]
 [0 0 1]
 [0 0 0]]
[[[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]]]
{'accuracy': 0.6666666666666665, 'precision': 0.0, 'f1': 0.0, 'tpr': 0.0, 'fpr': 0.0, 'shd': 3.0}



/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/minicond

{'accuracy': 0.6666666666666665,
 'precision': 0.0,
 'f1': 0.0,
 'tpr': 0.0,
 'fpr': 0.0,
 'shd': 3.0}

In [9]:
print("Linear")
path = "/home/lds/github/Causality-informed-Generation/code1/database/final_dataset/hypothetic/data/Hypothetic_v4_linear_v/"
main(image_dir = path,
     scene_name = "V4_V",
     strategy = "basic",)

main(image_dir = path,
     scene_name = "V4_V",
     strategy = "explicted",)

print("Nonlinear")
path = '/home/lds/github/Causality-informed-Generation/code1/database/final_dataset/hypothetic/data/Hypothetic_v4_nonlinear_v'
main(image_dir = path,
     scene_name = "V4_V",
     strategy = "basic",)

main(image_dir = path,
     scene_name = "V4_V",
     strategy = "explicted",)



Linear


100%|██████████| 15/15 [01:50<00:00,  7.40s/it]
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap

[[0 0 1 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 0 0]]
[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]]
{'accuracy': 0.8125, 'precision': 0.0, 'f1': 0.0, 'tpr': 0.0, 'fpr': 0.0, 'shd': 3.0

100%|██████████| 15/15 [01:52<00:00,  7.47s/it]
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap

[[0 0 1 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 0 0]]
[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]]
{'accuracy': 0.8166666666666667, 'precision': 0.06666666666666667, 'f1': 0.033333333

...

[[0 0 1 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 0 0]]
[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]]
{'accuracy': 0.8208333333333333, 'precision': 0.13333333333333333, 'f1': 0.066666666

100%|██████████| 15/15 [01:58<00:00,  7.92s/it]

[[0 0 1 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 0 0]]
[[[0, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]]
{'accuracy': 0.8083333333333333, 'precision': 0.1, 'f1': 0.060000000000000005, 'tpr'


/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/minicond

{'accuracy': 0.8083333333333333,
 'precision': 0.1,
 'f1': 0.060000000000000005,
 'tpr': 0.04444444444444444,
 'fpr': 0.015384615384615385,
 'shd': 3.066666666666667}

In [10]:
print("Linear")
path = "/home/lds/github/Causality-informed-Generation/code1/database/final_dataset/hypothetic/data/Hypothetic_V4_linear_full_connected/"
main(image_dir = path,
     scene_name = "V4_F",
     strategy = "basic",)

main(image_dir = path,
     scene_name = "V4_F",
     strategy = "explicted",)



Linear


100%|██████████| 15/15 [01:55<00:00,  7.69s/it]
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap

[[0 1 1 0]
 [0 0 1 0]
 [0 0 0 0]
 [1 0 1 0]]
[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]]
{'accuracy': 0.6875, 'precision': 0.0, 'f1': 0.0, 'tpr': 0.0, 'fpr': 0.0, 'shd': 5.0

100%|██████████| 15/15 [02:02<00:00,  8.18s/it]

[[0 1 1 0]
 [0 0 1 0]
 [0 0 0 0]
 [1 0 1 0]]
[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]]
{'accuracy': 0.6875, 'precision': 0.0, 'f1': 0.0, 'tpr': 0.0, 'fpr': 0.0, 'shd': 5.0


/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/minicond

{'accuracy': 0.6875,
 'precision': 0.0,
 'f1': 0.0,
 'tpr': 0.0,
 'fpr': 0.0,
 'shd': 5.0}

In [12]:
print("Linear")
path = "/home/lds/github/Causality-informed-Generation/code1/database/final_dataset/hypothetic/data/Hypothetic_v5_linear/"
main(image_dir = path,
     scene_name = "V5",
     strategy = "basic",)

main(image_dir = path,
     scene_name = "V5",
     strategy = "explicted",)

print("Nonlinear")
path = '/home/lds/github/Causality-informed-Generation/code1/database/final_dataset/hypothetic/data/Hypothetic_V5_nonlinear/'
main(image_dir = path,
     scene_name = "V5",
     strategy = "basic",)

main(image_dir = path,
     scene_name = "V5",
     strategy = "explicted",)



Linear


100%|██████████| 15/15 [02:04<00:00,  8.27s/it]
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap

[[0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 1 0 0 1]
 [0 0 0 0 0]]
[[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0]], [[0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0]], [[0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0

 20%|██        | 3/15 [00:23<01:33,  7.81s/it]

Error: invalid syntax (<string>, line 2)


...

Error: invalid syntax (<string>, line 2)


100%|██████████| 15/15 [02:18<00:00,  9.24s/it]

[[0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 1 0 0 1]
 [0 0 0 0 0]]
[[[0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0]], [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0]], [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0]], [[0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0


/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/minicond

{'accuracy': 0.8200000000000003,
 'precision': 0.4642857142857143,
 'f1': 0.30498866213151926,
 'tpr': 0.24285714285714285,
 'fpr': 0.03571428571428571,
 'shd': 4.5}

In [4]:
print("Linear")
path = "/home/lds/github/Causality-informed-Generation/code1/database/final_dataset/hypothetic/data/Hypothetic_v5_linear_full_connected/"
main(image_dir = path,
     scene_name = "V5_F",
     strategy = "basic",)

main(image_dir = path,
     scene_name = "V5_F",
     strategy = "explicted",)



Linear
dict_keys(['Reflection', 'Spring', 'Seesaw', 'seesaw', 'Magnets', 'Convex', 'Parabola', 'Waterflow', 'Pendulum', 'V2', 'V3_V', 'V3_F', 'V4_V', 'V4_F', 'V5', 'V5_F', 'P_reflection', 'P_refraction'])


  0%|          | 0/15 [00:00<?, ?it/s]

 20%|██        | 3/15 [00:28<01:52,  9.35s/it]

Error: Matrix not found in the response


 27%|██▋       | 4/15 [00:36<01:36,  8.73s/it]

...

[[0 1 1 0 1]
 [0 0 1 0 0]
 [0 0 0 1 1]
 [0 0 0 0 1]
 [0 0 0 0 0]]
[[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0]], [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0

100%|██████████| 15/15 [02:26<00:00,  9.74s/it]

[[0 1 1 0 1]
 [0 0 1 0 0]
 [0 0 0 1 1]
 [0 0 0 0 1]
 [0 0 0 0 0]]
[[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0


/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/miniconda3/envs/joe/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lds/minicond

{'accuracy': 0.7573333333333335,
 'precision': 0.6333333333333333,
 'f1': 0.264983164983165,
 'tpr': 0.17142857142857137,
 'fpr': 0.014814814814814814,
 'shd': 6.066666666666666}